# Gym demo

Demonstrate the use of the the Gym interface

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
import glob as glob
import time
import pickle
from omegaconf import OmegaConf

# add custom paths
sys.path.extend([os.path.abspath('./assetto_corsa_gym')])
import AssettoCorsaEnv.assettoCorsa as assettoCorsa

# Configure the logging system
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Format of the log messages
    datefmt='%Y-%m-%d %H:%M:%S',  # Format of the timestamp
)



## Load config file

In [2]:
config = OmegaConf.load("config.yml")

# Create env object

In [3]:
env = assettoCorsa.make_ac_env(cfg=config, work_dir="output")


INFO:AssettoCorsaEnv.ac_env:Setting track monza
INFO:AssettoCorsaEnv.track:Track loaded from: /home/mengst/Downloads/code/assetto_corsa_gym/assetto_corsa_gym/AssettoCorsaEnv/../AssettoCorsaConfigs/tracks/monza.csv Found 3760 segments downsampled by 10= 376 segments
INFO:AssettoCorsaEnv.reference_lap:Reference Lap. Loading: /home/mengst/Downloads/code/assetto_corsa_gym/assetto_corsa_gym/AssettoCorsaEnv/../AssettoCorsaConfigs/tracks/monza-racing_line.csv
INFO:AssettoCorsaEnv.reference_lap:Calculating curvature
INFO:AssettoCorsaEnv.sensors_ray_casting:spatial_hash len 60
/home/mengst/.conda/envs/p309/lib/python3.9/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
INFO:AssettoCorsaEnv.ac_env:Adding previous obs to state 25*3 = 100
INFO:AssettoCorsaEnv.ac_env:state_dim 125
INFO:AssettoCorsaEnv.ac_env:action_space: Box(-1.0, 1.0, (3,), float32)


# Show static info

In [4]:
static_info = env.client.simulation_management.get_static_info()
ac_mod_config = env.client.simulation_management.get_config()

logger.info("Static info:")
for i in static_info:
    logger.info(f"{i}: {static_info[i]}")
logger.info("AC Mod config:")
for i in ac_mod_config:
    logger.info(f"{i}: {ac_mod_config[i]}")


INFO:__main__:Static info:
INFO:__main__:TyreContactPoint_RR: [41.09868621826172, -10.244190216064453, 658.04541015625]
INFO:__main__:TrackLength: 5758.66064453125
INFO:__main__:penaltiesEnabled: 1
INFO:__main__:WindSpeed: 0
INFO:__main__:isCarInPitlane: 0
INFO:__main__:isCarInPit: 0
INFO:__main__:TrackConfiguration: 
INFO:__main__:WorldPosition: [41.79574966430664, -9.863253593444824, 659.4534301757812]
INFO:__main__:LastSplits: [0, 0, 0]
INFO:__main__:TyreContactPoint_FL: [42.484886169433594, -10.262252807617188, 660.7045288085938]
INFO:__main__:TrackName: monza
INFO:__main__:TyreContactPoint_FR: [40.8742790222168, -10.257800102233887, 660.5455322265625]
INFO:__main__:autoShifterOn: 1
INFO:__main__:CarName: bmw_z4_gt3
INFO:__main__:WindDirection: 359
INFO:__main__:CAR_WHEELBASE: 2.5102360499411227
INFO:__main__:CAR_WHEEL_R: [0.3400000035762787, 0.3400000035762787, 0.35249999165534973, 0.35249999165534973]
INFO:__main__:CAR_TRACK: 1.662847960865313
INFO:__main__:TyreContactPoint_RL: [

# Apply actions and recover the car

In [6]:
states = []

env.reset()

for i in range(100):
    if i % 2 == 0:
        steer = .1
    else:
        steer = -.1
    env.set_actions(np.array([steer, 0.1, -1.]))
    next_state, reward, done, info = env.step(action=None)  # action is already applied
    if done:
        break

env.recover_car()
env.close()

INFO:AssettoCorsaEnv.ac_env:Reset AC. Episode 2 total_steps: 103
INFO:AssettoCorsaEnv.ac_client:sending reset to simulation management server
INFO:AssettoCorsaEnv.ac_client:AC Client. Listening at host: localhost port: 2345
INFO:AssettoCorsaEnv.ac_client:Client connected on 2345
INFO:AssettoCorsaEnv.ac_env:Recover car
INFO:AssettoCorsaEnv.ac_client:sending reset to simulation management server
INFO:AssettoCorsaEnv.ac_env:Saved raw data to: output/laps//20250415_112211.574_states.parquet
INFO:AssettoCorsaEnv.ac_env:total_steps: 206 ep_steps: 100 ep_reward:    9.8 LapDist: 4873.97 packages lost 0 BestLap: 0.0
INFO:AssettoCorsaEnv.ac_env:LapNo_0:   0.00
INFO:AssettoCorsaEnv.ac_env:ep_bestLapTime:   0.00
INFO:AssettoCorsaEnv.ac_env:speed_mean:   8.14 speed_max:  22.31 max_abs_gap:   0.13 ep_laps: 1
INFO:AssettoCorsaEnv.ac_env:dt avr: 0.040 std: 0.001 min: 0.039 max: 0.042


{'ep_count': 2,
 'ep_steps': 101,
 'total_steps': 206,
 'packages_lost': 0,
 'ep_reward': 9.810763080597944,
 'speed_mean': 8.136283050853722,
 'speed_max': 22.314556121826172,
 'BestLap': 0.0,
 'terminated': False,
 'LapNo_0': 0.0,
 'ep_bestLapTime': 0}

# Plot states

In [ ]:
df = env.get_history()
df.columns

In [ ]:
plt.title("Speed")
plt.plot(df.speed)
plt.show()

plt.title("steerAngle")
plt.plot(df.steerAngle)
plt.show()